# Tracing Basics

### The OpenAI Problem
I used Claude instead of OpenAI.

### Setup

Make sure you set your environment variables, including your OpenAI API key.

In [1]:
# You can set them inline
import os
os.environ["OPENAI_API_KEY"] = ""
os.environ["LANGSMITH_API_KEY"] = ""
os.environ["LANGSMITH_TRACING"] = "true"
os.environ["LANGSMITH_PROJECT"] = "langsmith-academy"

In [1]:
# Or you can use a .env file
import os
from dotenv import load_dotenv
load_dotenv(dotenv_path="../../../.env", override=True)
os.environ["USER_AGENT"] = "496"
os.environ["TOKENIZERS_PARALLELISM"] = "false"


In [2]:
import warnings
import numpy as np

# Suppress sklearn warnings
warnings.filterwarnings('ignore', category=RuntimeWarning, module='sklearn')
np.seterr(divide='ignore', invalid='ignore', over='ignore')

{'divide': 'warn', 'over': 'warn', 'under': 'ignore', 'invalid': 'warn'}

### Tracing with @traceable

The @traceable decorator is a simple way to log traces from the LangSmith Python SDK. Simply decorate any function with @traceable.

The decorator works by creating a run tree for you each time the function is called and inserting it within the current trace. The function inputs, name, and other information is then streamed to LangSmith. If the function raises an error or if it returns a response, that information is also added to the tree, and updates are patched to LangSmith so you can detect and diagnose sources of errors. This is all done on a background thread to avoid blocking your app's execution.

In [3]:
# Imports
from langsmith import traceable
from anthropic import Anthropic
from typing import List
import nest_asyncio
from utils import get_vector_db_retriever

# Configuration
MODEL_PROVIDER = "anthropic"
MODEL_NAME = "claude-sonnet-4-20250514"  # or "claude-3-5-sonnet-20241022"
APP_VERSION = 1.0
RAG_SYSTEM_PROMPT = """You are an assistant for question-answering tasks.
Use the following pieces of retrieved context to answer the latest question in the conversation.
If you don't know the answer, just say that you don't know.
Use three sentences maximum and keep the answer concise.
"""

# Initialize
anthropic_client = Anthropic()
nest_asyncio.apply()
retriever = get_vector_db_retriever()

# Traced Functions
@traceable(metadata={"vectordb": "sklearn"})
def retrieve_documents(question: str):
    return retriever.invoke(question)

@traceable
def generate_response(question: str, documents):
    formatted_docs = "\n\n".join(doc.page_content for doc in documents)
    messages = [
        {
            "role": "user",
            "content": f"Context: {formatted_docs} \n\n Question: {question}"
        }
    ]
    return call_claude(messages)

@traceable(
    metadata={"model_name": MODEL_NAME, "model_provider": MODEL_PROVIDER}
)
def call_claude(
    messages: List[dict],
    model: str = MODEL_NAME,
    temperature: float = 0.0
) -> str:
    """Call Anthropic's Claude API"""
    response = anthropic_client.messages.create(
        model=model,
        max_tokens=1024,
        system=RAG_SYSTEM_PROMPT,  # System message is separate in Anthropic
        messages=messages,
        temperature=temperature,
    )
    return response.content[0].text

@traceable
def langsmith_rag(question: str):
    documents = retrieve_documents(question)
    response = generate_response(question, documents)
    return response

@traceable handles the RunTree lifecycle for you!

In [4]:
question = "How can I trace with the @traceable decorator?"
ai_answer = langsmith_rag(question)
print(ai_answer)

To trace with the @traceable decorator, you need to follow these steps:

First, set the required environment variables: `LANGSMITH_TRACING='true'` and `LANGSMITH_API_KEY` to your API key. Simply import the decorator from langsmith and add `@traceable` above any function you want to trace: `from langsmith import traceable` then `@traceable` before your function definition. When calling a sync function wrapped with @traceable, use the `await` keyword to ensure the trace is logged correctly.


##### Let's take a look in LangSmith!

### Adding Metadata

LangSmith supports sending arbitrary metadata along with traces.

Metadata is a collection of key-value pairs that can be attached to runs. Metadata can be used to store additional information about a run, such as the version of the application that generated the run, the environment in which the run was generated, or any other information that you want to associate with a run. Similar to tags, you can use metadata to filter runs in the LangSmith UI, and can be used to group runs together for analysis.

In [5]:
from langsmith import traceable

@traceable(
    metadata={"vectordb": "sklearn"}
)
def retrieve_documents(question: str):
    return retriever.invoke(question)

@traceable
def generate_response(question: str, documents):
    formatted_docs = "\n\n".join(doc.page_content for doc in documents)
    messages = [
        {
            "role": "user",
            "content": f"Context: {formatted_docs} \n\n Question: {question}"
        }
    ]
    return call_claude(messages)

@traceable(
    metadata={"model_name": MODEL_NAME, "model_provider": MODEL_PROVIDER}
)
def call_claude(
    messages: List[dict], model: str = MODEL_NAME, temperature: float = 0.0
) -> str:
    response = anthropic_client.messages.create(
        model=model,
        max_tokens=1024,
        system=RAG_SYSTEM_PROMPT,  # System message is separate in Anthropic
        messages=messages,
        temperature=temperature,
    )
    return response.content[0].text

@traceable
def langsmith_rag(question: str):
    documents = retrieve_documents(question)
    response = generate_response(question, documents)
    return response  # Already returns text, not a response object

You can also add metadata at runtime!

In [8]:
question = "How do I add metadata at runtime?"
ai_answer = langsmith_rag(question, langsmith_extra={"metadata": {"runtime_metadata": "foo"}})
print(ai_answer)

You can add metadata at runtime by passing it in the configuration parameters when invoking your chain. Specifically, you include a `metadata` dictionary in the second parameter of the `invoke()` method, like this: `chain.invoke({"input": "What is the meaning of life?"}, {"metadata": {"invoke-key": "invoke-value"}})`. This runtime metadata will be associated with that specific invocation and can be used for tracking and filtering runs in LangSmith.


##### Let's take a look in LangSmith!